# Blackholes Models and Grids

Synthesizer has a collection of routines allowing us to also model the spectral energy distributions of black holes (i.e. AGN) and combine their emission with the emission of others galaxy components (i.e. stellar emission).

Like the stellar emission this is enabled through blackhole grids which contain blackhole spectra over a grid of parameters.

Currently we have grids created for two blackhole models: the **Cloudy AGN model** and the **Feltre et al. (2016)** model.

## Models

### Cloudy AGN model


The default blackhole model used by `synthesizer` is the one built into the `cloudy` photoionisation code. This is a two-component model consisting of a "big bump" component with a rising power-law and high-energy exponential cutoff. It is parameterised by the temperature of the bump ($T_{\rm BB}$), the X-ray to UV ratio $\alpha_{\rm ox}$, the low-energy slope of the Big Bump continuum $\alpha_{\rm uv}$, and the slope of the X-ray component $\alpha_{x}$. The full continuum is described by,

$ f_{\nu} = \nu^{\alpha_{UV}} \; exp\left (\frac{-h\nu}{kT_{\rm BB}} \right )\,exp\left (\frac{-kT_{\rm IR}}{h\nu} \right ) + a\nu^{\alpha_{X}} $

Here $a$ is a coefficient determined to produce the desired $\alpha_{\rm ox}$. For most grids we fix $\alpha_{\rm ox}$, $\alpha_{\rm x}$, and $\alpha_{\rm uv}$ to the default values suggested by the `cloudy` documentation ($\alpha_{\rm ox}=-1.4$, $\alpha_{\rm x}=-1$, and $\alpha_{\rm uv}=-0.5$). 

The big-bump temperature $T_{BB}$ is related to the blackhole mass $M_{\bullet}$ and accretion $\dot{M}_{\bullet}$ rate through:

$ T_{BB} = \left ( \frac{3c^{6}}{8\pi6^{3}\sigma_{SB}G^{2}} \frac{\dot{M}_{\bullet}}{M^{2}_{\bullet}} \right )^{1/4}  = 2.24 \times 10^{9} \left ( \frac{\dot{M}_{\bullet}}{M_{\odot}\,yr^{-1}} \right )^{1/4} \left ( \frac{M_{\bullet}}{M_{\odot}} \right )^{-1/2} $

and hence it is left as an axis in all grids.

For ease of use we make available several grids, differing in the amount of parameters that are varied:

- `blackholes_cloudy_c17.03_log10TBB`: only the big-bump temperature is varied. This grid is included as part of the test suite with the name `test_grid_blackholes`.
- `blackholes_cloudy_c17.03_log10TBB_log10Z`: both the big-bump temperature and gas-phase metallicity are varied.


### Feltre et al. (2016) AGN NLR model

*Needs to be added*

## Using the Grids

Like stellar grids, the blackhole spectra are accessed using the `synthesizer`'s  `Grid` functionality, e.g.:  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmasher as cmr
import h5py
import os
from unyt import c, Angstrom
from synthesizer.grid import Grid

In [ ]:
grid_dir =  '../../../tests/test_grid/'
grid_name = 'test_grid_blackholes'


hf = h5py.File('../../../tests/test_grid/test_grid_blackholes.hdf5', "r")

hf.visit(print)


In [ ]:
grid = Grid(grid_name=grid_name, grid_dir=grid_dir)

In [ ]:
print(grid)

Let's plot the spectra for different temperatures.

In [ ]:

spectra_type = 'incident'
spectra_type = 'total'

cmap = cmr.bubblegum
norm = mpl.colors.Normalize(vmin=4., vmax=7.) 

# initialise plot
fig = plt.figure(figsize=(3.5, 5.))

left = 0.15
height = 0.8
bottom = 0.1
width = 0.8

# define main ax
ax = fig.add_axes((left, bottom, width, height))

# define colourbar ax
cax = fig.add_axes((left, bottom+height, width, 0.02))

# add colourbar
cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, orientation='horizontal')  # add the colourbar

# colourbar formatting and labelling
cax.xaxis.tick_top()
cax.xaxis.set_label_position('top')
cax.set_xlabel(r'$\rm \log_{10}(T/K)$')

# loop over log10ages
for i, log10T in enumerate(grid.log10T):
    
    # get spectra
    Lnu = grid.spectra[spectra_type][i, :]
    # Lnu = fnu_to_flam(grid.lam, Lnu)

    # plot spectra
    ax.plot(np.log10(grid.lam), np.log10(Lnu), c=cmap(norm(log10T)), lw=1, alpha=0.8)

# plot Lyman and Balmer limits for reference
for wv in [912., 3646.]:
    ax.axvline(np.log10(wv), c='k', lw=1, alpha=0.5)

# set wavelength range (log(Angstrom))
ax.set_xlim([1., 5.])

# set luminosity range
ax.set_ylim([-20, -13])

# add labels
ax.set_xlabel(r'$\rm log_{10}(\lambda/\AA)$')
ax.set_ylabel(r'$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$')

#### Bolometric correction

Another thing we can do is calculate the bolometric correction.

To do this let's extract an `Sed` object from the `Grid` object since that provides some helpful methods:



In [ ]:
grid_point = grid.get_grid_point((6.,))
print(grid_point)

sed = grid.get_sed(grid_point)
print(sed)

Lbol = sed.measure_bolometric_luminosity()
print(Lbol)

Luv = np.interp(1500., sed.lam, sed.lnu) * (c / (1500 * Angstrom)).to('Hz').value
print(Luv)

print(Lbol/Luv)


print(np.interp(1500., sed.lam, sed.lnu)/Lbol)


The bolometric correction by Hopkins et al. (2007) is:

In [ ]:
def kappa(Lbol):
    return 6.25*(Lbol/1E10)**-0.37 + 9*(Lbol/1E10)**-0.012


print(kappa(1E14))